**Info on status of this notebook - 27/01/23**

Code snippets below open U and V on pressure levels, compute S and store in annual chunks. 

This needs cleanup and transfer to preprocess_JLU.py

But first, the metadata issues flagged below and mentioned in #5 on Github need to be addressed. Need Merja for this. If these issues are not solved, I will potentially have to ignore this model. 

In [2]:
import xarray as xr
data_path = "../data/JLU/"

In [3]:
# content of params
horizontal_ranges = {
    "rlons": slice(-28.21, 17.99),
    "rlats": slice(-23.21, 21.67)
}

EXPERIMENTS = ["GRASS", "FOREST", "EVAL"]


In [4]:
experiment="EVAL"

In [5]:
variable="U"

In [6]:
pressure_levels={"EVAL": [750, 850], 
                 "GRASS": [750, 850, 925], 
                 "FOREST": [750, 850, 925]}

In [47]:
ds_list = []
for pressure_level in pressure_levels[experiment]:
    ds_u = xr.open_mfdataset(data_path + experiment + "/U/U"+str(pressure_level)+"*.nc")
    ds_v = xr.open_mfdataset(data_path + experiment + "/V/V"+str(pressure_level)+"*.nc")
    ds_list.append(xr.merge([ds_u, ds_v]))

In [50]:
ds_list[1]

<xarray.Dataset>
Dimensions:       (time: 43828, pressure: 1, rlat: 109, rlon: 112)
Coordinates:
    lat           (rlat, rlon) float32 dask.array<chunksize=(109, 112), meta=np.ndarray>
    lon           (rlat, rlon) float32 dask.array<chunksize=(109, 112), meta=np.ndarray>
  * pressure      (pressure) float32 8.5e+04
  * rlat          (rlat) float32 -24.53 -24.09 -23.65 ... 22.11 22.55 22.99
  * rlon          (rlon) float32 -29.53 -29.09 -28.65 ... 18.43 18.87 19.31
  * time          (time) datetime64[ns] 1986-01-01 ... 2015-12-31T18:00:00
Data variables:
    U             (time, pressure, rlat, rlon) float32 dask.array<chunksize=(1460, 1, 109, 112), meta=np.ndarray>
    rotated_pole  (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b'' b''
    V             (time, pressure, rlat, rlon) float32 dask.array<chunksize=(1460, 1, 109, 112), meta=np.ndarray>
Attributes: (12/22)
    title:                          CLMcom-CCLM5-0-9 model output prepared fo...
    institution:                    Chair of Environmental Meteorlogy, Brande...
    source:                         Climate Limited-area Modelling Community ...
    project_id:                     CORDEX
    experiment_id:                  LUCAS_Eval
    realization:                    5
    ...                             ...
    institute_id:                   BTU
    model_id:                       BTU-CCLM5-0-9
    rcm_version_id:                 v1
    CORDEX_domain:                  EUR-44
    product:                        output
    btu_run_id:                     FPSL01

In [35]:
ds["S"] = (ds["U"] ** 2 + ds["V"] ** 2) ** (1.0 / 2)

# Drop non-needed variables and save to file
ds = ds.drop(["U", "V"])

In [46]:
ds_u

<xarray.Dataset>
Dimensions:       (time: 43828, pressure: 1, rlat: 109, rlon: 112)
Coordinates:
    lat           (rlat, rlon) float32 dask.array<chunksize=(109, 112), meta=np.ndarray>
    lon           (rlat, rlon) float32 dask.array<chunksize=(109, 112), meta=np.ndarray>
  * pressure      (pressure) float32 8.5e+04
  * rlat          (rlat) float32 -24.53 -24.09 -23.65 ... 22.11 22.55 22.99
  * rlon          (rlon) float32 -29.53 -29.09 -28.65 ... 18.43 18.87 19.31
  * time          (time) datetime64[ns] 1986-01-01 ... 2015-12-31T18:00:00
Data variables:
    U             (time, pressure, rlat, rlon) float32 dask.array<chunksize=(1460, 1, 109, 112), meta=np.ndarray>
    rotated_pole  (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b'' b''
Attributes: (12/22)
    title:                          CLMcom-CCLM5-0-9 model output prepared fo...
    institution:                    Chair of Environmental Meteorlogy, Brande...
    source:                         Climate Limited-area Modelling Community ...
    project_id:                     CORDEX
    experiment_id:                  LUCAS_Eval
    realization:                    5
    ...                             ...
    institute_id:                   BTU
    model_id:                       BTU-CCLM5-0-9
    rcm_version_id:                 v1
    CORDEX_domain:                  EUR-44
    product:                        output
    btu_run_id:                     FPSL01

In [42]:
year="1986"
ds_tmp = ds.sel({"time":year})

In [43]:
ds.to_netcdf(data_path + experiment + "/S/S" + str(year) + ".nc")

## Metadata issues

### EVAL

In [68]:
filename = data_path + experiment + "/U/U750p_1986010100-1986123118.nc"
filename

'../data/JLU/EVAL/U/U750p_1986010100-1986123118.nc'

In [69]:
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:       (time: 1460, pressure: 1, rlat: 109, rlon: 112)
Coordinates:
    lat           (rlat, rlon) float32 ...
    lon           (rlat, rlon) float32 ...
  * pressure      (pressure) float32 8.5e+04
  * rlat          (rlat) float32 -24.53 -24.09 -23.65 ... 22.11 22.55 22.99
  * rlon          (rlon) float32 -29.53 -29.09 -28.65 ... 18.43 18.87 19.31
  * time          (time) datetime64[ns] 1986-01-01 ... 1986-12-31T18:00:00
Data variables:
    U             (time, pressure, rlat, rlon) float32 ...
    rotated_pole  |S1 ...
Attributes: (12/22)
    title:                          CLMcom-CCLM5-0-9 model output prepared fo...
    institution:                    Chair of Environmental Meteorlogy, Brande...
    source:                         Climate Limited-area Modelling Community ...
    project_id:                     CORDEX
    experiment_id:                  LUCAS_Eval
    realization:                    5
    ...                             ...
    institute_id:                   BTU
    model_id:                       BTU-CCLM5-0-9
    rcm_version_id:                 v1
    CORDEX_domain:                  EUR-44
    product:                        output
    btu_run_id:                     FPSL01

## FOREST

In [63]:
filename = data_path + "FOREST/U/U925p_FOREST_1980-2016.nc"
filename

'../data/JLU/FOREST/U/U925p_FOREST_1980-2016.nc'

In [64]:
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:       (time: 54061, bnds: 2, rlon: 112, rlat: 109, pressure: 1)
Coordinates:
  * time          (time) datetime64[ns] 1980-01-01 ... 2017-01-01
  * rlon          (rlon) float32 -29.53 -29.09 -28.65 ... 18.43 18.87 19.31
  * rlat          (rlat) float32 -24.53 -24.09 -23.65 ... 22.11 22.55 22.99
    lon           (rlat, rlon) float32 ...
    lat           (rlat, rlon) float32 ...
  * pressure      (pressure) float32 9.5e+04
Dimensions without coordinates: bnds
Data variables:
    time_bnds     (time, bnds) datetime64[ns] ...
    U             (time, pressure, rlat, rlon) float32 ...
    rotated_pole  |S1 ...
Attributes:
    title:           COSMO5.0_CLM9 simulation (0.44 Deg) with ERA-Interim for...
    institution:     JLU (Justus-Liebig-University, Germany
    source:          CCLM_5-0-9.exe
    project_id:      Flagship Pilot Study (FPS) LUCAS
    experiment_id:   LUCAS_I_Forest
    realization:     5
    Conventions:     CF-1.4
    conventionsURL:  http://www.cfconventions.org/
    contact:         Merja Toelle (merja.toelle@geogr.uni-giessen.de}
    references:      http://www.clm-community.eu/
    creation_date:   2017-10-06 17:58:08

**FOREST has its own different problem. Filename says 925p but metadata says 950**

### GRASS

In [66]:
filename = data_path + "GRASS/U/U925p_GRASS_1980-2016.nc"
filename

'../data/JLU/GRASS/U/U925p_GRASS_1980-2016.nc'

In [67]:
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:       (time: 54061, bnds: 2, rlon: 112, rlat: 109, pressure: 1)
Coordinates:
  * time          (time) datetime64[ns] 1980-01-01 ... 2017-01-01
  * rlon          (rlon) float32 -29.53 -29.09 -28.65 ... 18.43 18.87 19.31
  * rlat          (rlat) float32 -24.53 -24.09 -23.65 ... 22.11 22.55 22.99
    lon           (rlat, rlon) float32 ...
    lat           (rlat, rlon) float32 ...
  * pressure      (pressure) float32 9.5e+04
Dimensions without coordinates: bnds
Data variables:
    time_bnds     (time, bnds) datetime64[ns] ...
    U             (time, pressure, rlat, rlon) float32 ...
    rotated_pole  |S1 ...
Attributes:
    title:           COSMO5.0_CLM9 simulation (0.44 Deg) with ERA-Interim for...
    institution:     JLU (Justus-Liebig-University, Germany
    source:          CCLM_5-0-9.exe
    project_id:      Flagship Pilot Study (FPS) LUCAS
    experiment_id:   LUCAS_I_Forest
    realization:     5
    Conventions:     CF-1.4
    conventionsURL:  http://www.cfconventions.org/
    contact:         Merja Toelle (merja.toelle@geogr.uni-giessen.de}
    references:      http://www.clm-community.eu/
    creation_date:   2017-10-06 21:18:38

**same prob as for FOREST**